# Пиявка - Python-based completion

Kludgier than we expected because we had to hack a somewhat reliable data source here.

---
## Setup

In [ ]:
from bs4 import BeautifulSoup

import csv
import json
import datetime

---
## Make CSV from HTML file

These are not well-formed HTML files; they're the table snippets scraped off the BNO spreadsheet.  Each `table-XX.html` file contains only the data table and additional headers.

In [33]:
soup = BeautifulSoup(open('./site-data/table-01.html').read(), 'html.parser')

In [35]:
table = soup.find('table')

outputRows  = list()
rowCount    = 0
IGNORE_ROWS = 5
for tableRow in table.find_all('tr'):
    rowCount += 1
    
    if rowCount > IGNORE_ROWS:
        # print(tableRow)
        columns = tableRow.find_all('td')
        row = list()
        for column in columns:
            row.append(column.text.replace(',', ''))
    
        nixed = ('Diamond Princess', 'Queue', 'TOTAL',)
        if any(s in row for s in nixed) or not len(row[0]):
            continue

        outputRows.append(row)

outputRows

[['UNITED STATES',
  'Cases',
  'Deaths',
  'Serious',
  'Critical',
  'Recovered',
  'Links',
  ''],
 ['Washington', '267', '24', '', '', '1', 'Source', ''],
 ['New York', '173', '0', '', '', '0', 'Source', ''],
 ['California', '109', '3', '', '1', '1', 'Source', ''],
 ['Massachusetts', '92', '0', '', '', '', 'Source', ''],
 ['Georgia', '22', '0', '', '', '', 'Source', ''],
 ['Florida', '21', '2', '', '', '', 'Source', ''],
 ['Illinois', '19', '0', '', '', '2', 'Source', ''],
 ['New Jersey', '15', '1', '', '', '', 'Source', ''],
 ['Oregon', '15', '0', '', '', '', 'Source', ''],
 ['Pennsylvania', '11', '0', '', '1', '', 'Source', ''],
 ['South Carolina', '9', '0', '', '', '', 'Source', ''],
 ['Colorado', '8', '0', '', '', '', 'Source', ''],
 ['Kentucky', '8', '0', '', '', '', 'Source', ''],
 ['Texas', '7', '0', '', '', '', 'Source', ''],
 ['Maryland', '6', '0', '0', '0', '0', 'Source', ''],
 ['Arizona', '5', '0', '', '', '1', 'Source', ''],
 ['Virginia', '5', '0', '', '', '', 'Source',

In [20]:
with open('./site-data/scraped-world.tsv', 'w') as outputFile:
    writer = csv.writer(outputFile, delimiter = '\t')
    writer.writerows(outputRows)

---
## World

In [21]:
rawSource = 'site-data/scraped-world.tsv'

updateWorld = dict()
today  = datetime.date.today().strftime("%m-%d-%Y")
with open(rawSource, 'r') as inputFile:
    rawData = csv.DictReader(inputFile, delimiter = '\t')
    for row in rawData:
        if 'Diamond' not in row['OTHER PLACES']:
                updateWorld[row['OTHER PLACES']] = { today: float(row['Cases'])}

if 'Queue' in updateWorld:
    del(updateWorld['Queue'])

updateWorld['US'] = updateWorld['United States']
del(updateWorld['United States'])
updateWorld['UK'] = updateWorld['United Kingdom']
del(updateWorld['United Kingdom'])
updateWorld['Denmark'] = updateWorld['Denmark*']
del(updateWorld['Denmark*'])
updateWorld['United Arab Emirates'] = updateWorld['UAE']
del(updateWorld['UAE'])
updateWorld['Bosnia and Herzegovina'] = updateWorld['Bosnia']
del(updateWorld['Bosnia'])
# UK same thing
updateWorld['US']

{'03-11-2020': 1016.0}

In [22]:
with open('site-data/confirmed.json', 'r') as inputFile:
    dataWorld = json.load(inputFile)

# Not in the list
dataWorld['Turkey'] = { today: 0.0, }

In [24]:
for country in updateWorld.keys():
    dataWorld[country][today] = updateWorld[country][today]

dataWorld

{'!Global': {'01/22/2020': 555.0,
  '01/23/2020': 653.0,
  '01/24/2020': 941.0,
  '01/25/2020': 1434.0,
  '01/26/2020': 2118.0,
  '01/27/2020': 2927.0,
  '01/28/2020': 5578.0,
  '01/29/2020': 6166.0,
  '01/30/2020': 8234.0,
  '01/31/2020': 9927.0,
  '02/01/2020': 12038.0,
  '02/02/2020': 16787.0,
  '02/03/2020': 19881.0,
  '02/04/2020': 23892.0,
  '02/05/2020': 27635.0,
  '02/06/2020': 30817.0,
  '02/07/2020': 34391.0,
  '02/08/2020': 37120.0,
  '02/09/2020': 40150.0,
  '02/10/2020': 42762.0,
  '02/11/2020': 44802.0,
  '02/12/2020': 45221.0,
  '02/13/2020': 60368.0,
  '02/14/2020': 66885.0,
  '02/15/2020': 69030.0,
  '02/16/2020': 71224.0,
  '02/17/2020': 73258.0,
  '02/18/2020': 75136.0,
  '02/19/2020': 75639.0,
  '02/20/2020': 76197.0,
  '02/21/2020': 76823.0,
  '02/22/2020': 78579.0,
  '02/23/2020': 78965.0,
  '02/24/2020': 79568.0,
  '02/25/2020': 80413.0,
  '02/26/2020': 81395.0,
  '02/27/2020': 82754.0,
  '02/28/2020': 84120.0,
  '02/29/2020': 86011.0,
  '03/01/2020': 88369.0,
  

---
## United States

In [ ]:
rawSource = 'site-data/scraped-US.tsv'

updateUS = dict()

with open(rawSource, 'r') as inputFile:
    rawData = csv.DictReader(inputFile, delimiter = "\t")
    for row in rawData:
        updateUS[row['UNITED STATES']] = { today: float(row['Cases'])}

updateUS['Washington D.C.'] = updateUS['District of Columbia']
del(updateUS['District of Columbia'])

In [ ]:
updateUS

In [ ]:
with open('site-data/confirmed-US.json', 'r') as inputFile:
    dataUS = json.load(inputFile)

dataUS['Grand Princess'] = { today: 0.0, }

In [ ]:
for state in updateUS.keys():
    dataUS[state][today] = updateUS[state][today]

dataUS['California']

---
## US Regions


In [ ]:
from covidvu.vujson import US_REGIONS_LONG

In [ ]:
with open('site-data/confirmed-US.json', 'r') as inputFile:
    dataUSRegions = json.load(inputFile)

updateUSRegions = dict()

for state in updateUS:
    if US_REGIONS_LONG[state] not in updateUSRegions:
        updateUSRegions[US_REGIONS_LONG[state]] = { today: 0.0 }
    
    updateUSRegions[US_REGIONS_LONG[state]][today] += float(updateUS[state][today])

updateUSRegions

---
## Calculate total US cases from JH CSSE data

In [ ]:
from covidvu.vujson import JH_CSSE_FILE_CONFIRMED
import pandas as pd

cases = pd.read_csv(JH_CSSE_FILE_CONFIRMED)
cases.groupby('Country/Region').sum().T['US'].tail()

In [ ]:
cases = pd.read_csv(JH_CSSE_FILE_CONFIRMED)
cases

---
&#169; the COVIDvu Contributors.  All rights reserved